In [105]:
almanac = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""



def parse_almanac(almanac):
    mappings = dict()
    for block in almanac.split('\n\n'):
        if "seeds:" in block:
            seeds = [int(x) for x in block.split(': ')[1].split()]
            continue
        lines = block.split('\n')
        mappings[lines[0][:-5]] = [[int(n) for n in l.split(" ")] for l in lines[1:]]
        
    return seeds, mappings

seeds, mappings = parse_almanac(almanac)
seeds, mappings

([79, 14, 55, 13],
 {'seed-to-soil': [[50, 98, 2], [52, 50, 48]],
  'soil-to-fertilizer': [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
  'fertilizer-to-water': [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
  'water-to-light': [[88, 18, 7], [18, 25, 70]],
  'light-to-temperature': [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
  'temperature-to-humidity': [[0, 69, 1], [1, 0, 69]],
  'humidity-to-location': [[60, 56, 37], [56, 93, 4]]})

In [106]:
def create_mapper(mapping):
    def mapper(x):
        for d,s,l in mapping:
            if  s <= x < s+l:
                return x + d - s
        return x 
    return mapper


seed_mappers = [create_mapper(mapping) for mapping in mappings.values()]

seed = 79
for mapper in seed_mappers:
    seed = mapper(seed)
    
print(seed)

82


In [107]:
def get_locations(seeds, mappings):
    mappers = [create_mapper(mapping) for mapping in mappings.values()]
    for mapper in mappers:
        seeds = list(map(mapper, seeds))
    return seeds

locations = get_locations(seeds, mappings)
locations


[82, 43, 86, 35]

In [108]:
def create_reverse(mapping):
    def mapper(y):
        for d,s,l in mapping:
            if d <= y < d+l:
                return y - d + s
        return y 
    return mapper

mappers = [create_reverse(mapping) for mapping in mappings.values()]
def get_seed(location):
    for mapper in reversed(mappers):
        location = mapper(location)
    return location

def check_if_in_range(seed):
    for start, l in zip(seeds[::2],seeds[1::2]):
        if start <= seed < start+l:
            return True
    return False

def find_smallest():
    location = 0
    while True:
        seed = get_seed(location)
        if check_if_in_range(seed):
            return location
        location += 1
find_smallest()

46

In [109]:
with open("i5") as f:
    almanac = f.read()
    
seeds, mappings = parse_almanac(almanac)
locations = get_locations(*parse_almanac(almanac))
min(locations)

510109797

In [127]:
def create_reverse(mapping):
    def mapper(y):
        for d,s,l in mapping:
            if d <= y < d+l:
                return y - d + s
        return y 
    return mapper

mappers = [create_reverse(mapping) for mapping in mappings.values()]
def get_seed(location):
    for mapper in reversed(mappers):
        location = mapper(location)
    return location

def check_if_in_range(seed):
    for start, l in zip(seeds[::2],seeds[1::2]):
        if start <= seed < start+l:
            return True
    return False

def is_valid_location(location):
    return check_if_in_range(get_seed(location))



In [ ]:

location_ranges = mappings["humidity-to-location"]
ranges = [[s,s+l] for s,_,l in location_ranges]
ranges
seed_ranges = []
for i, (s,e) in enumerate(ranges):
    if is_valid_location(s):
        seed_ranges.append((s,i))
    if is_valid_location(e):
        seed_ranges.append((e,i))

sorted(seed_ranges) # to find the closest bin

[(367508399, 14),
 (367508399, 16),
 (740912129, 8),
 (740912129, 14),
 (1068860974, 8),
 (1525482420, 1),
 (1525482420, 5),
 (2327084890, 3),
 (2327084890, 11),
 (2422798960, 9),
 (2422798960, 11),
 (3854006626, 2),
 (3854006626, 4),
 (4029880359, 0),
 (4029880359, 12)]

must be in between here in the last chuck

In [129]:
is_valid_location(0) , check_if_in_range(get_seed(367508399))

(False, True)

In [155]:
def find_closest(start=0, end=367508399):
    """if you know the right answer is somewhere in a range
    i remember that its very fast to just half the space and 
    check the remaining half
    so instead of checking N, whats the maximum checks?
    each check halves the space, so how many steps from N to 1?
    2^x = N : start at the solution, x doublings to reach space N
    -> x ~ log(n) -> O(log N) which is fast i guess ? :) 
    because log2(367508399) ~ 29 steps
    .. ahh. its called *binary search*
    """
    mid = (start + end) // 2
    print(start, end, mid)
    if start == end-1:
        print("exiting", end)
        return end
    if is_valid_location(mid):
        return find_closest(start, mid)
    else:
        return find_closest(mid, end)
        
found = find_closest()
print(f"{found-1} is valid? {is_valid_location(found-1)}")
print(f"Minimum {found=} is valid? {is_valid_location(found)}")


0 367508399 183754199
0 183754199 91877099
0 91877099 45938549
0 45938549 22969274
0 22969274 11484637
0 11484637 5742318
5742318 11484637 8613477
8613477 11484637 10049057
8613477 10049057 9331267
9331267 10049057 9690162
9331267 9690162 9510714
9510714 9690162 9600438
9600438 9690162 9645300
9600438 9645300 9622869
9600438 9622869 9611653
9611653 9622869 9617261
9617261 9622869 9620065
9620065 9622869 9621467
9621467 9622869 9622168
9622168 9622869 9622518
9622518 9622869 9622693
9622518 9622693 9622605
9622605 9622693 9622649
9622605 9622649 9622627
9622605 9622627 9622616
9622616 9622627 9622621
9622621 9622627 9622624
9622621 9622624 9622622
9622621 9622622 9622621
exiting 9622622
9622621 is valid? False
Minimum found=9622622 is valid? True
